In [1]:
import os
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import math as mt
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import warnings
import itertools
import datetime
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import pandas as pd
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
sns.set()
import warnings
warnings.filterwarnings('ignore')
path_store='C:/Users/chuna/Downloads/week5/store.csv'
path_train='C:/Users/chuna/Downloads/week5/train.csv'
path_test='C:/Users/chuna/Downloads/week5/test.csv'
isFile = os.path.isfile(path_train) 
print(isFile)                           

df_store = pd.read_csv(path_store)
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)



#df_train=df_train.sample(frac =.000005)


True


In [3]:
df_train["Date"]= pd.to_datetime(df_train["Date"]) 
df_train["Date"]=df_train["Date"].astype('datetime64[D]')
holydates=df_train.loc[((df_train['StateHoliday'] != '0'))]['Date'].unique()


In [4]:
def nearest(pivot):
    return min(holydates, key=lambda x: abs(x - pivot))

In [5]:
def convert2int(df_train):

    df_train['Store'] = df_train['Store'].astype(int)
    df_train['DayOfWeek'] = df_train['DayOfWeek'].astype(int)
    
    
    df_train['Open'] = df_train['Open'].astype(int)
    df_train['Promo'] = df_train['Promo'].astype(int)
    df_train['SchoolHoliday'] = df_train['SchoolHoliday'].astype(int)
    return df_train

In [6]:
def convert2date(df_train):

    df_train["Date"]= pd.to_datetime(df_train["Date"]) 
    df_train["Date"]=df_train["Date"].astype('datetime64[D]')
    holydates=df_train.loc[((df_train['StateHoliday'] != '0'))]['Date'].unique()
    return df_train

In [7]:
def new_insights(df_train):

    df_train['day'] = df_train['Date'].dt.day_name() 
    df_train['month'] = df_train['Date'].dt.month_name() 
    df_train['year'] = df_train['Date'].dt.year  
    df_train.loc[((df_train['day'] == 'Saturday') |(df_train['day'] == 'Sunday')  ) , 'weekday'] = 'weekend' 
    df_train['weekday']=df_train['weekday'].fillna('weekday')
    df_train['StateHoliday'] = df_train['StateHoliday'].astype(str)
    


    df_train['Difference_to_StateHoliday'] = df_train['Date'].apply(nearest)

    df_train['Difference_to_StateHoliday'] = df_train['Date'].sub(df_train['Difference_to_StateHoliday'], axis=0)

    return df_train

In [8]:
def new_insights2(df_train):

    resort=df_train.copy()
    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(str)

    resort["Difference_to_StateHoliday"]= resort["Difference_to_StateHoliday"].str.split("d", 1)
    resort['Difference_to_StateHoliday'] = resort["Difference_to_StateHoliday"].str.get(0) 
    resort.sample(3)
    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(str)

    resort['Difference_to_StateHoliday'] =resort.Difference_to_StateHoliday.mask(resort.Difference_to_StateHoliday.str[-1].isin(['-','+']), resort.Difference_to_StateHoliday.str[-1].str.cat(resort.Difference_to_StateHoliday.str[:-1])).astype('float')


    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(int)
    df_train=resort
    return df_train

In [9]:

# read the training data set
data = df_train

# top rows of the data
data.head()

# seperate the independent and target variables
train_x = data.drop(columns=['Sales'])
train_y = data['Sales']

In [10]:
# import the BaseEstimator
from sklearn.base import BaseEstimator

# define the class OutletTypeEncoder
# This will be our custom transformer that will create 3 new binary columns
# custom transformer must have methods fit and transform
class OutletTypeEncoder(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        
        x_dataset=convert2int(x_dataset)
        x_dataset=convert2date(x_dataset)
        x_dataset=new_insights(x_dataset)
        x_dataset=new_insights2(x_dataset)
        

        return x_dataset

In [11]:
df_train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [12]:
# pre-processsing step
# Drop the columns - 
# Impute the missing values in column Item_Weight by mean
# Scale the data in the column Item_MRP
pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[('drop_columns', 'drop', ['Date',
                                                                        'Customers' ]),
                                                                        
                                                                      
                                              ('scale_data', StandardScaler(),['Difference_to_StateHoliday']),
                                              
                                               ('encoder', OneHotEncoder(), ['Promo','StateHoliday','SchoolHoliday','day','month','weekday','Store','year']) ])
                                             

In [13]:
# Define the Pipeline
"""
Step1: get the oultet binary columns
Step2: pre processing
Step3: Train a Random Forest Model
"""
model_pipeline = Pipeline(steps=[('get_outlet_binary_columns', OutletTypeEncoder()), 
                                 ('pre_processing',pre_process),
                                 ('random_forest', RandomForestRegressor(max_depth=10,random_state=2))
                                 ])
# fit the pipeline with the training data
model_pipeline.fit(train_x,train_y)

# predict target values on the training data
random_forst=model_pipeline.predict(train_x)

In [15]:
predict_train=model_pipeline.predict(train_x)c

SyntaxError: invalid syntax (<ipython-input-15-12d4e7bad2c7>, line 1)

In [17]:
from sklearn.metrics import mean_squared_error
print('RMSE on train data: ', mean_squared_error(train_y, random_forst)**(0.5))

RMSE on train data:  2330.857635495658


In [29]:
train_x

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,day,month,year,weekday,Difference_to_StateHoliday
0,1,5,2015-07-31,555,1,1,0,1,Friday,July,2015,weekday,57 days
1,2,5,2015-07-31,625,1,1,0,1,Friday,July,2015,weekday,57 days
2,3,5,2015-07-31,821,1,1,0,1,Friday,July,2015,weekday,57 days
3,4,5,2015-07-31,1498,1,1,0,1,Friday,July,2015,weekday,57 days
4,5,5,2015-07-31,559,1,1,0,1,Friday,July,2015,weekday,57 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,a,1,Tuesday,January,2013,weekday,0 days
1017205,1112,2,2013-01-01,0,0,0,a,1,Tuesday,January,2013,weekday,0 days
1017206,1113,2,2013-01-01,0,0,0,a,1,Tuesday,January,2013,weekday,0 days
1017207,1114,2,2013-01-01,0,0,0,a,1,Tuesday,January,2013,weekday,0 days


In [31]:
train_y

0           5263
1           6064
2           8314
3          13995
4           4822
           ...  
1017204        0
1017205        0
1017206        0
1017207        0
1017208        0
Name: Sales, Length: 1017209, dtype: int64

In [32]:
random_forst[1700:1745]

array([7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123, 7629.51295123, 7629.51295123, 7629.51295123,
       7629.51295123])

In [20]:
plt.figure(figsize=(10,7))
feat_importances = pd.Series(model_pipeline.feature_importances_, index = train_x.columns)
feat_importances.nlargest(7).plot(kind='barh');

AttributeError: 'Pipeline' object has no attribute 'feature_importances_'

<Figure size 720x504 with 0 Axes>

In [36]:
x=model_pipeline.steps[2][1].feature_importances_

In [40]:
x[:19]

array([1.14554214e-02, 5.00014177e-02, 6.62317481e-02, 4.58332413e-04,
       7.38208550e-06, 1.03534576e-06, 2.91335496e-06, 5.85917502e-05,
       4.29633654e-05, 4.20997106e-05, 1.34312188e-02, 2.83239142e-04,
       4.94443226e-04, 5.28546276e-06, 3.59128803e-05, 1.50076205e-05,
       3.41866610e-05, 1.34853747e-05, 1.84317556e-02])

In [42]:
pip install eli5


Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda install -c conda-forge eli5

In [ ]:
onehot_columns = list(model_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['one_hot'].get_feature_names(input_features=categorical_features))
numeric_features_list = list(numeric_features)
numeric_features_list.extend(onehot_columns)